In [13]:
import json
from pydantic import BaseModel
from openai import OpenAI
from persona import Persona 
from docx import Document

client = OpenAI()

# Get what the objective of the focus group is
objective = "I want to know what people think about Emory University students."
 
# Ask if there are restrictions
restriction = "I want to only recruit people who exaggerate"

class Archetypes(BaseModel):
    archetype1: str 
    archetype2: str
    archetype3: str
    archetype4: str
    archetype5: str
    archetype6: str
    archetype7: str
    archetype8: str
    
class Persona(BaseModel):
    fullname: str
    age: str
    race_ethnicity: str
    education_level: str
    income: str
    personality: str
    priorities: str
    values: str
    
class Questions(BaseModel):
    opening_question: str
    key_question1: str
    key_question2: str
    key_question3: str
    key_question4: str
    key_question5: str
    key_question6: str
    ending_question: str

# Decide what 8 personalities/demographics will be in the focus group
def generateListOfArchetypes(objective, restriction):
    
    prompt = [{"role": "system", "content": '''You are an expert on focus groups who excels at building and recruiting ideal focus groups. 
                                                Given the following objective and restrictions, give 8 ideal archetypes of individuals to be recruited for the focus group'''},
            {"role": "user", "content": f'''Objective: {objective}.
                                            Restriction: {restriction}'''}]
    
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=prompt,
        response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "archetypes",
            "schema": {
                "type": "object",
                "properties": {
                    "archetype1": {"type": "string"},
                    "archetype2": {"type": "string"},
                    "archetype3": {"type": "string"},
                    "archetype4": {"type": "string"},
                    "archetype5": {"type": "string"},
                    "archetype6": {"type": "string"},
                    "archetype7": {"type": "string"},
                    "archetype8": {"type": "string"},
                },
                "required": ["archetype1", "archetype2", "archetype3", "archetype4", "archetype5", "archetype6", "archetype7", "archetype8"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
    )
   
    return prompt, json.loads(completion.choices[0].message.content)


# Create a personality
def generatePersonas(prompt: str, archetypes: dict): 
    persona_schema = {
        "type": "object",
        "properties": {
            "fullname": {"type": "string"},
            "age": {"type": "integer"},
            "race": {"type": "string"},            
            "ethnicity": {"type": "string"},
            "nationality": {"type": "string"},
            "education_level": {"type": "string"},
            "income": {"type": "integer"},
            "talking_style": {"type": "string"},
            "short_bio": {"type": "string"},
            "priorities": {"type": "string"},
            "values": {"type": "string"}
        },
        "required": ["fullname", "age", "race", "ethnicity", "nationality", "education_level", "income", "talking_style", "short_bio", "priorities", "values"],
        "additionalProperties": False
    }

    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": '''Based on the previous chat and archetype given, create a diverse focus group with 8 personas that is both realistic and feasible.'''},
            {"role": "user", "content": f'''Previous Chat: {prompt}
                                            Archetype: {archetypes}'''}
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "archetypes",
                "schema": {
                    "type": "object",
                    "properties": {
                        f"persona{i+1}": persona_schema for i in range(8)
                    },
                    "required": [f"persona{i+1}" for i in range(8)],
                    "additionalProperties": False
                },
                "strict": True
            }
        }
    )
   
    return json.loads(completion.choices[0].message.content)

# Come up with Focus Group Questions
def generateQuestions(objective: str):
    prompt = [
        {
            "role": "system", 
            "content": '''You are an expert on focus groups who excels at building and recruiting ideal focus groups. 
                          Given the following objective, come up with 1 opening question, 6 key questions, and 1 ending question.'''
        },
        {
            "role": "user", 
            "content": f'''Objective: {objective}.'''
        }
    ]
    
    # Make the API call to generate questions
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=prompt,
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "focus_group_questions",
                "schema": {
                    "type": "object",
                    "properties": {
                        "opening_question": {"type": "string"},
                        "key_question1": {"type": "string"},
                        "key_question2": {"type": "string"},
                        "key_question3": {"type": "string"},
                        "key_question4": {"type": "string"},
                        "key_question5": {"type": "string"},
                        "key_question6": {"type": "string"},
                        "ending_question": {"type": "string"}
                    },
                    "required": [
                        "opening_question", "key_question1", "key_question2", "key_question3", "key_question4", "key_question5", "key_question6", "ending_question"
                    ],
                    "additionalProperties": False
                },
                "strict": True
            }
        }
    )
    
    return json.loads(completion.choices[0].message.content)

# Generate answer to questions given a persona
def generateAnswers(persona, questions):
    prompt = [
        {
            "role": "system", 
            "content": '''You are given the following persona for a focus group. Embody the persona you are given and answer all the questions comprehensively that is true to who you are.'''
        },
        {
            "role": "user", 
            "content": f'''Persona: {persona}
                            Questions: {questions}'''
        }
    ]
    
    completion = client.beta.chat.completions.parse(
        model="gpt-4o",
        temperature=1.0,
        messages=prompt,
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "focus_group_answers",
                "schema": {
                    "type": "object",
                    "properties": {
                        "opening_answer": {"type": "string"},
                        "key_answer1": {"type": "string"},
                        "key_answer2": {"type": "string"},
                        "key_answer3": {"type": "string"},
                        "key_answer4": {"type": "string"},
                        "key_answer5": {"type": "string"},
                        "key_answer6": {"type": "string"},
                        "ending_answer": {"type": "string"}
                    },
                    "required": [
                        "opening_answer", "key_answer1", "key_answer2", "key_answer3", "key_answer4", "key_answer5", "key_answer6", "ending_answer"
                    ],
                    "additionalProperties": False
                },
                "strict": True
            }
        }
    )
    
    answers = json.loads(completion.choices[0].message.content)
    with open(f"data/{persona['fullname']}.txt", "w") as file:
        persona_qa = ""
        for answer_key in answers:
            answer = answers[answer_key]
            question_key = answer_key.replace("answer", "question")
            question = questions[question_key]
            persona_qa += f"Q: {question}\nA: {answer}\n\n"
        file.write(persona_qa)
        file.close()
    
    
# Generate Report for each question based on the personality 
def generateReport(objective, personas):
    complete_transcript = ""
    for key in personas:
        persona = personas[key]
        complete_transcript += str(persona)
        with open(f"data/{persona['fullname']}.txt", "r") as f:
            complete_transcript += f.read() 
        f.close()
        
    prompt = [
        {
            "role": "system", 
            "content": '''Write a full and complete comprehensive focus group report based on the focus group objective and complete transcript.'''
        },
        {
            "role": "user", 
            "content": f'''Objective: {objective}
                            Complete Transcript: {complete_transcript}'''
        }
    ]
    
    completion = client.beta.chat.completions.parse(
        model="gpt-4o",
        temperature=1.0,
        messages=prompt,
    )
    
    with open("Full Report.txt", "w") as file:
        file.write(completion.choices[0].message.content)
    file.close()
        
def generateReportDocument():
    with open("Full Report.txt", "r") as f:
        report = f.read() 
        
    document = Document()
    
    assistant = client.beta.assistants.create(
    name="python-docx Expert",
    instructions="You are a coding expert with python-docx. You will make direct edits to the Document to fulfill user requests. Make sure to save the document at the end",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4o",
    )
    
    thread = client.beta.threads.create()

    message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f'''I want you to format the following text prettily into 'document' which has already been declared 'document = Document()'.
                Text: {report}'''
    )
    
    run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
    )
    
    return run.status
    
    


# Allow interaction with each participant



In [14]:
message, archetypes = generateListOfArchetypes(objective, restriction)
print(message)
print(archetypes)

[{'role': 'system', 'content': 'You are an expert on focus groups who excels at building and recruiting ideal focus groups. \n                                                Given the following objective and restrictions, give 8 ideal archetypes of individuals to be recruited for the focus group'}, {'role': 'user', 'content': 'Objective: I want to know what people think about Emory University students..\n                                            Restriction: I want to only recruit people who exaggerate'}]
{'archetype1': 'A recent high school graduate who perceives all college students as extremely wealthy and privileged, believing they have it easy compared to others.', 'archetype2': 'A local business owner who thinks Emory students are always too focused on socializing to take their studies seriously, claiming they never see students work hard.', 'archetype3': 'A parent of a high school student who believes that all Emory students are destined to become CEOs and influential leaders,

In [15]:
personas = generatePersonas(message, archetypes)

for key in personas:
    print(personas[key])

{'fullname': 'Jessica Tran', 'age': 18, 'race': 'Asian', 'ethnicity': 'Vietnamese', 'nationality': 'American', 'education_level': 'High School Graduate', 'income': 20000, 'talking_style': 'Idealistic and dramatic', 'short_bio': 'A recent high school graduate from Atlanta, Jessica believes that all college students, especially those at Emory, have it easy. She often exaggerates the wealth and privilege of these students, claiming they are spoiled.', 'priorities': 'Education, social justice', 'values': 'Equality, opportunity'}
{'fullname': 'Mark Wilson', 'age': 45, 'race': 'White', 'ethnicity': 'Caucasian', 'nationality': 'American', 'education_level': 'Real Estate License', 'income': 75000, 'talking_style': 'Sarcastic and blunt', 'short_bio': 'As a local coffee shop owner, Mark thinks Emory students are more interested in social outings than studies. He often jokes that they spend all their time partying instead of preparing for life after school.', 'priorities': 'Business growth, commu

In [16]:
questions = generateQuestions(objective)

for key in questions:
    print(questions[key])

What are your initial thoughts or perceptions when you hear about Emory University and its students?
How would you describe the academic environment at Emory University based on what you know or have experienced?
What characteristics do you associate with Emory University students, both academically and socially?
Have you had any personal experiences with Emory students, and how have those experiences shaped your opinion of them?
In what ways do you think Emory University students differ from those at other universities, particularly within the region?
What do you think are the strengths and weaknesses of Emory University students in comparison to students from other institutions?
How do your perceptions of Emory students align with or differ from what you hear in the media or from others?
If you had to summarize your overall view of Emory University students in a few words or a short statement, what would it be?


In [17]:
for key in personas:
    generateAnswers(personas[key], questions)

In [18]:
generateReport(objective, personas)

In [5]:
generateReportDocument()

'completed'

### Focus Group Report on Perceptions of Emory University Students

#### Objective:
The objective of this focus group was to explore and understand perceptions of Emory University students from diverse community members. This insight could guide future engagement strategies, improve community relations, and influence prospective student outreach programs.

#### Focus Group Participants:
The focus group comprised a diverse range of participants in terms of age, background, profession, and perspectives. Participants included recent high school graduates, local business owners, parents with aspirations for their children, retired professionals, community dwellers, and alumni from less prestigious institutions.

#### Key Findings:
1. **Perceived Privilege and Elitism**:
   - Participants like Jessica Tran and Carlos Ramirez highlighted a perception of privilege among Emory students. Jessica emphasized the resources and opportunities they have, often painting a picture of entitlement. Carlos, as a concerned community member, noted that students act detached from the local community and tradition.
   - Michael Johnson and Elaine Thompson, criticized the perceived elitism, suggesting that students at Emory prioritize social status over real-world engagement.

2. **Academic Excellence vs. Community Engagement**:
   - Dr. Robert Smith and Linda Grayson recognized academic rigor and intelligence, emphasizing Emory students’ preparedness for leadership roles and global perspectives. 
   - However, most participants, such as Mark Wilson and Samantha Lee, commented on how this focus on academic success might overshadow real-world issues and societal engagement. 

3. **Social Dynamics**:
   - The balance (or imbalance) between academia and social life was a significant point of discussion, with Mark Wilson and Samantha Lee noting the vibrant social scenes and party culture of Emory students.
   - This duality was often cited as a strength, promoting a well-rounded experience, but also a potential risk factor for missing community connection.

4. **Impact on Local Community**:
   - Participants voiced concerns about the impact of Emory students on the local community. Carlos Ramirez expressed frustration over what he perceives as an erosion of community charm and a lack of engagement.
   - Elaine Thompson's views aligned similarly, noting a disconnect between Emory students and real-world community issues.

5. **Media vs. Personal Interaction**:
   - There was a contrast between media portrayals and personal experiences. While media often highlights the academic and leadership successes of Emory students, personal interactions revealed a perceived lack of humility and community awareness. Michael Johnson, for example, felt the media missed the day-to-day perceptions of elitism experienced by those outside the Emory bubble.

#### Recommendations:
1. **Community Engagement Initiatives**:
   - Emory University should consider bolstering community engagement initiatives, creating programs that encourage students to participate more actively in local community development and social justice projects.

2. **Cultural and Empathy Training**:
   - Implement workshops focusing on cultural awareness and empathy to help students better connect with and understand diverse community backgrounds, addressing the concerns raised by Carlos Ramirez and Elaine Thompson.

3. **Enhanced Communication Channels**:
   - Establish open forums and dialogue sessions between students and community members to foster understanding and mitigate perceptions of elitism.

4. **Promote Balanced Portrayals**:
   - Encourage media portrayals that capture the full spectrum of Emory student life, including their contributions to community and lesser-known efforts in societal engagement.

5. **Alumni and Community Member Initiatives**:
   - Engage alumni from other institutions, like Michael Johnson, in mentorship roles, offering diverse perspectives that could redefine success and overcome biases towards institutional prestige.

#### Conclusion:
The focus group insights reveal a dichotomy between the academic prowess of Emory students and their perceived societal engagement. While recognized for their intellect and ambitious nature, fostering closer ties with the local community could enhance their overall contribution and mitigate existing biases of privilege and elitism. These findings should inform strategic planning for Emory University’s community relations and student development initiatives.